# Final Models

In this notebook we look at some final candidate models and assess their performance on a validation set

In [47]:
import json
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')
plt.style.use('seaborn-darkgrid')
%matplotlib inline

random.seed(17)

### Loading data

We will subsample 50,000 of the 200,000 records for training by sampling 25% of the data from each of the five music categories

In [48]:
data = None
with open(os.path.join('data', 'train.json'), 'r') as train_file:
    data = [json.loads(row) for row in train_file]

In [49]:
data_df = pd.DataFrame(data).drop(columns=['image'])
del data

In [50]:
categories = data_df['category'].unique()
dfs = []
for category in categories:
    dfs.append(data_df[data_df['category'] == category].sample(frac=0.25))
data_df = pd.concat(dfs, axis=0)
data_df = data_df.sort_index()
data_df

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash
0,4.0,"08 24, 2010",u04428712,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,85559980
5,5.0,"09 7, 2015",u07624734,o.k.,Five Stars,1441584000,Pop,$14.99,p78489708,23609516
11,5.0,"11 16, 2006",u40719083,Katchen's performace throughout this collectio...,Superb interpretations by Katchen,1163635200,Classical,$31.04,p63362921,40704096
14,5.0,"08 12, 2016",u66989964,She is my fave!,Five Stars,1470960000,Pop,$11.57,p83852395,05580669
21,5.0,"10 31, 2016",u34469863,Really Great,Really Great,1477872000,Pop,$7.98,p51677500,28306462
...,...,...,...,...,...,...,...,...,...,...
199985,4.0,"12 4, 2014",u65640560,Had to have in my collection..,Four Stars,1417651200,Jazz,$2.15,p76891834,67429536
199989,5.0,"12 28, 2010",u92755262,I first heard Taking Back Sunday this summer w...,Great CD,1293494400,Alternative Rock,$12.97,p50896575,46693447
199994,4.0,"09 3, 2014",u97165206,I love this lp this album is really good\none ...,great song,1409702400,Pop,$49.99,p58216418,07085315
199995,4.0,"05 1, 2004",u68902609,"With this, Mariah's third album, Mariah proved...",Well Done Mariah! You Show 'Em!,1083369600,Pop,$7.98,p84118731,35077372


### Pre-Processing

We apply feature cleaning as prototyped before and then split into a training and validation set, ensuring that the proportion of data points in training vs validation is consistent for each music category

In [51]:
def trim_price(price):
    """Trims `price` to remove the $ sign.
    
    If the price variable does not have the format $x.xx
    then the empty string is returned.
    
    Parameters
    ----------
    price: str
        A string representing a price.
    
    Returns
    -------
    str
        A string representing `price` but with the $ sign removed,
        or the empty string if `price` does not have the correct
        format.
    
    """
    if (not pd.isnull(price) and isinstance(price, str) and
        len(price) > 0 and price[0] == '$'):
        return price[1:]
    return ""

In [52]:
from datetime import datetime

data_df['reviewMonth'] = data_df['reviewTime'].apply(lambda x: x.split(' ')[0])
data_df['reviewYear'] = data_df['reviewTime'].apply(lambda x: x.split(' ')[2])
data_df['reviewHour'] = data_df['unixReviewTime'].apply(lambda x: datetime.fromtimestamp(x).hour)
data_df['reviewMonthYear'] = data_df['reviewYear'] + '-' + data_df['reviewMonth']

data_df['cleanedPrice'] = data_df['price'].apply(lambda x: trim_price(x))
data_df = data_df[data_df['cleanedPrice'] != ""]
data_df['cleanedPrice'] = data_df['cleanedPrice'].astype('float')

data_df['fixedReviewText'] = np.where(pd.isnull(data_df['reviewText']), "", data_df['reviewText'])
data_df['fixedSummary'] = np.where(pd.isnull(data_df['summary']), "", data_df['summary'])
data_df['fullReviewText'] = data_df['fixedSummary'] + " " + data_df['fixedReviewText']

data_df = data_df.drop(columns=['fixedReviewText', 'fixedSummary'])

genres = data_df['category'].unique()

for genre in genres:
    genre_col = "is" + genre.replace(" ", "").replace("&", "")
    data_df[genre_col] = data_df['category'].apply(lambda x: 1 if x == genre else 0)

data_df['reviewWordCount'] = data_df['fullReviewText'].apply(lambda x: len(x.split()))

data_df

<ipython-input-52-d3ba8e2e1b50>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['cleanedPrice'] = data_df['cleanedPrice'].astype('float')
<ipython-input-52-d3ba8e2e1b50>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['fixedReviewText'] = np.where(pd.isnull(data_df['reviewText']), "", data_df['reviewText'])
<ipython-input-52-d3ba8e2e1b50>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,...,reviewHour,reviewMonthYear,cleanedPrice,fullReviewText,isPop,isClassical,isDanceElectronic,isAlternativeRock,isJazz,reviewWordCount
0,4.0,"08 24, 2010",u04428712,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,85559980,...,20,2010-08,35.93,Amazing that I Actually Bought This...More Ama...,1,0,0,0,0,277
5,5.0,"09 7, 2015",u07624734,o.k.,Five Stars,1441584000,Pop,$14.99,p78489708,23609516,...,20,2015-09,14.99,Five Stars o.k.,1,0,0,0,0,3
11,5.0,"11 16, 2006",u40719083,Katchen's performace throughout this collectio...,Superb interpretations by Katchen,1163635200,Classical,$31.04,p63362921,40704096,...,19,2006-11,31.04,Superb interpretations by Katchen Katchen's pe...,0,1,0,0,0,226
14,5.0,"08 12, 2016",u66989964,She is my fave!,Five Stars,1470960000,Pop,$11.57,p83852395,05580669,...,20,2016-08,11.57,Five Stars She is my fave!,1,0,0,0,0,6
21,5.0,"10 31, 2016",u34469863,Really Great,Really Great,1477872000,Pop,$7.98,p51677500,28306462,...,20,2016-10,7.98,Really Great Really Great,1,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199985,4.0,"12 4, 2014",u65640560,Had to have in my collection..,Four Stars,1417651200,Jazz,$2.15,p76891834,67429536,...,19,2014-12,2.15,Four Stars Had to have in my collection..,0,0,0,0,1,8
199989,5.0,"12 28, 2010",u92755262,I first heard Taking Back Sunday this summer w...,Great CD,1293494400,Alternative Rock,$12.97,p50896575,46693447,...,19,2010-12,12.97,Great CD I first heard Taking Back Sunday this...,0,0,0,1,0,43
199994,4.0,"09 3, 2014",u97165206,I love this lp this album is really good\none ...,great song,1409702400,Pop,$49.99,p58216418,07085315,...,20,2014-09,49.99,great song I love this lp this album is really...,1,0,0,0,0,16
199995,4.0,"05 1, 2004",u68902609,"With this, Mariah's third album, Mariah proved...",Well Done Mariah! You Show 'Em!,1083369600,Pop,$7.98,p84118731,35077372,...,20,2004-05,7.98,"Well Done Mariah! You Show 'Em! With this, Mar...",1,0,0,0,0,172


In [53]:
def calculate_MSE(actuals, predicteds):
    """Calculates the Mean Squared Error between `actuals` and `predicteds`.
    
    Parameters
    ----------
    actuals: np.array
        A numpy array of the actual values.
    predicteds: np.array
        A numpy array of the predicted values.
    
    Returns
    -------
    float
        A float representing the Mean Squared Error between `actuals` and
        `predicteds`.
    
    """
    return (((actuals - predicteds)**2).sum()) / (len(actuals))

In [54]:
from sklearn.model_selection import train_test_split

genres = data_df['category'].unique()
X_train_set = []
X_val_set = []
y_train_set = []
y_val_set = []

for genre in genres:
    genre_df = data_df[data_df['category'] == genre]
    targets = genre_df['overall']
    feature_data = genre_df.drop(columns=['overall'])
    X_train, X_val, y_train, y_val = train_test_split(
        feature_data, targets, shuffle=True, test_size=0.2, random_state=17)
    X_train_set.append(X_train)
    X_val_set.append(X_val)
    y_train_set.append(y_train)
    y_val_set.append(y_val)

X_train = pd.concat(X_train_set)
X_val = pd.concat(X_val_set)
y_train = pd.concat(y_train_set)
y_val = pd.concat(y_val_set)

### Collaborative Filtering

In this model we only need a users ID, the items ID, and 

In [55]:
def user_item_matrix(df, rating_col, user_col, item_col):
    return sp.csr_matrix(df[rating_col], (df[user_col], df[item_col]))

In [56]:
train_data = pd.concat([X_train, pd.DataFrame(y_train, columns=['overall'])], axis=1)
val_data = pd.concat([X_val, pd.DataFrame(y_val, columns=['overall'])], axis=1)

In [57]:
import scipy.sparse as sp

item_matrix = train_data.pivot(index='itemID', columns='reviewerID', values='overall')
item_matrix = item_matrix.fillna(0)
user_item_train_matrix = sp.csr_matrix(item_matrix.values)

In [58]:
global_average = train_data['overall'].mean()

In [59]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(user_item_train_matrix)
item_neighbors = np.asarray(model_knn.kneighbors(user_item_train_matrix, return_distance=False))

In [60]:
user_matrix = train_data.pivot(index='reviewerID', columns='itemID', values='overall')
user_matrix = user_matrix.fillna(0)
user_item_train_matrix = sp.csr_matrix(user_matrix.values)

In [61]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(user_item_train_matrix)
user_neighbors = np.asarray(model_knn.kneighbors(user_item_train_matrix, return_distance=False))

In [62]:
train_user_avg = train_data.groupby(train_data['reviewerID'], as_index=False)['overall'].mean()
train_item_avg = train_data.groupby(train_data['itemID'], as_index=False)['overall'].mean()
train_user_avg.columns = ['reviewerID', 'userAverage']
train_item_avg.columns = ['itemID', 'itemAverage']
train_user_avg = train_user_avg.set_index('reviewerID')
train_item_avg = train_item_avg.set_index('itemID')

In [63]:
item_avgs = []
for i in range(len(item_neighbors)):
    item_avgs.append(train_item_avg['itemAverage'][item_matrix.index[item_neighbors[i]]].mean())

item_avgs = pd.concat([pd.DataFrame(item_matrix.index, columns=['itemID']), pd.DataFrame(item_avgs, columns=['itemRating'])], axis=1)

In [64]:
user_avgs = []
for i in range(len(user_neighbors)):
    user_avgs.append(train_user_avg['userAverage'][user_matrix.index[user_neighbors[i]]].mean())

user_avgs = pd.concat([pd.DataFrame(user_matrix.index, columns=['reviewerID']), pd.DataFrame(user_avgs, columns=['userRating'])], axis=1)

In [65]:
def weighted_average_data(X, total_avg, user_avgs, item_avgs):
    """Calculates the error based on the weighted average prediction.
    
    Parameters
    ----------
    X: pd.DataFrame
        The DataFrame of features.
    y: np.array
        A numpy array containing the targets
    total_avg: float
        The average across all users/items.
    user_avgs: pd.DataFrame
        A DataFrame containing the average rating for each user.
    item_avgs: pd.DataFrame
        A DataFrame containing the average rating for each item.
    
    Returns
    -------
    float
        A float representing the mean squared error of the predictions.
    
    """
    df_user = pd.merge(X, user_avgs, how='left', on=['reviewerID'])
    df_final = pd.merge(df_user, item_avgs, how='left', on=['itemID'])
    df_final = df_final[['userRating', 'itemRating']]
    df_final = df_final.fillna(total_avg)
    df_final.index = X.index
    return df_final

In [66]:
X_train_aug = weighted_average_data(X_train, global_average, user_avgs, item_avgs)
X_val_aug = weighted_average_data(X_val, global_average, user_avgs, item_avgs)

In [67]:
X_train_mod = pd.concat([X_train, X_train_aug], axis=1)
X_val_mod = pd.concat([X_val, X_val_aug], axis=1)

In [68]:
def threshold_rating(rating):
    """Thresholds `rating` to lie in the range [1, 5].
    
    Parameters
    ----------
    rating: float
        The rating to be thresholded.
    
    Returns
    -------
    float
        A float representing the thresholded rating.
    
    """
    if rating < 1:
        return 1
    if rating > 5:
        return 5
    return rating

In [69]:
X_train_mod['pred'] = (0.5 * X_train_mod['userRating']) + (0.5 * X_train_mod['itemRating'])
X_train_mod['pred'].apply(lambda x: threshold_rating(x))
print("Training MSE: {}".format(calculate_MSE(y_train, X_train_mod['pred'])))

X_val_mod['pred'] = (0.5 * X_val_mod['userRating']) + (0.5 * X_val_mod['itemRating'])
X_val_mod['pred'].apply(lambda x: threshold_rating(x))
print("Validation MSE: {}".format(calculate_MSE(y_val, X_val_mod['pred'])))

Training MSE: 0.8121797657358844
Validation MSE: 0.9497614288949675


### Language Models

In [98]:
columns_to_keep = ['cleanedPrice', 'isPop', 'isAlternativeRock', 'isJazz', 'isClassical', 'isDanceElectronic', 'reviewWordCount']
X_train_reg1 = X_train[columns_to_keep]
X_val_reg1 = X_val[columns_to_keep]

In [99]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train_reg1['reviewWordCount'] = X_train_reg1['reviewWordCount'].apply(lambda x: np.log(x))
X_val_reg1['reviewWordCount'] = X_val_reg1['reviewWordCount'].apply(lambda x: np.log(x))

<ipython-input-99-2b5d7d7878ed>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_reg1['reviewWordCount'] = X_train_reg1['reviewWordCount'].apply(lambda x: np.log(x))
<ipython-input-99-2b5d7d7878ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_reg1['reviewWordCount'] = X_val_reg1['reviewWordCount'].apply(lambda x: np.log(x))


In [100]:
def clean_dataset(df):
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

X_train_reg1 = clean_dataset(X_train_reg1)
y_train1 = y_train[y_train.index.isin(X_train_reg1.index)]
X_train1 = X_train[X_train.index.isin(X_train_reg1.index)]

X_val_reg1 = clean_dataset(X_val_reg1)
y_val1 = y_val[y_val.index.isin(X_val_reg1.index)]
X_val1 = X_val[X_val.index.isin(X_val_reg1.index)]

<ipython-input-100-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-100-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [101]:
X_train_mod = X_train_mod[X_train_mod.index.isin(X_train_reg1.index)]
X_val_mod = X_val_mod[X_val_mod.index.isin(X_val_reg1.index)]

In [102]:
X_train_reg1 = min_max_scaler.fit_transform(X_train_reg1)
X_val_reg1 = min_max_scaler.transform(X_val_reg1)

In [103]:
from sklearn.linear_model import Ridge

vthreshold_rating = np.vectorize(threshold_rating)

alphas = [0.0, 0.01, 0.03, 0.1, 0.3]
for alpha in alphas:
    print("Alpha = {}".format(alpha))
    print("------------")
    reg_model = Ridge(alpha=alpha)
    reg_model.fit(X_train_reg1, y_train1)
    print("Training Error: {}".format(calculate_MSE(y_train1, vthreshold_rating(reg_model.predict(X_train_reg1)))))
    print("Validation Error: {}".format(calculate_MSE(y_val1, vthreshold_rating(reg_model.predict(X_val_reg1)))))
    print()

Alpha = 0.0
------------
Training Error: 0.9588535445992827
Validation Error: 0.9262869599431929

Alpha = 0.01
------------
Training Error: 0.9588428957078677
Validation Error: 0.9262350113636552

Alpha = 0.03
------------
Training Error: 0.9588433477027621
Validation Error: 0.9262349757449848

Alpha = 0.1
------------
Training Error: 0.9588449304068484
Validation Error: 0.926234860637202

Alpha = 0.3
------------
Training Error: 0.9588494564503761
Validation Error: 0.9262346102237373



In [76]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def process_review_text(review_text, exclude_text, ps):
    """Pre-processes the text given by `review_text`.
    
    Parameters
    ----------
    review_text: str
        The review text to be processed.
    exclude_text: collection
        A collection of words to be excluded.
    ps: PorterStemmer
        The PorterStemmer used to perform word stemming.
    
    Returns
    -------
    str
        A string representing the processed version of `review_text`.
    
    """
    review = re.sub('[^a-zA-Z0-9]', ' ', review_text).lower().split()
    review = [ps.stem(word) for word in review if not word in exclude_text]
    return ' '.join(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Matthew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
X_train1['processedReview'] = X_train1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
X_val1['processedReview'] = X_val1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))

<ipython-input-77-0d22f415ca83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train1['processedReview'] = X_train1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
<ipython-input-77-0d22f415ca83>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val1['processedReview'] = X_val1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))


In [78]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X_train_cv1 = cv.fit_transform(X_train1['processedReview'])
X_val_cv1 = cv.transform(X_val1['processedReview'])

In [79]:
import scipy.sparse as sp

X_train_reg1_sp = sp.csr_matrix(X_train_reg1)
X_train_cv_reg1 = sp.hstack((X_train_cv1, X_train_reg1_sp), format='csr')

X_val_reg1_sp = sp.csr_matrix(X_val_reg1)
X_val_cv_reg1 = sp.hstack((X_val_cv1, X_val_reg1_sp), format='csr')

In [80]:
from xgboost import XGBRegressor

learning_rates = [0.01, 0.03, 0.1, 0.3, 0.5]
estimators = [50, 100, 200, 500]
depths = [1, 2, 5, 10]

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("--------------------------------------------------")
            xg_reg = XGBRegressor(
                learning_rate=learning_rate, max_depth=depth, n_estimators=estimator)
            xg_reg.fit(X_train_cv_reg1, y_train1)
            print("Training Error: {}".format(calculate_MSE(y_train1, vthreshold_rating(xg_reg.predict(X_train_cv_reg1)))))
            print("Validation Error: {}".format(calculate_MSE(y_val1, vthreshold_rating(xg_reg.predict(X_val_cv_reg1)))))
            print()

Learning Rate: 0.01, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 6.561420677223477
Validation Error: 6.583459713622337

Learning Rate: 0.01, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 6.539023898229513
Validation Error: 6.564129017293043

Learning Rate: 0.01, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 6.484453433139949
Validation Error: 6.517187858800406

Learning Rate: 0.01, # Estimators: 50, Depth: 10
--------------------------------------------------
Training Error: 6.411167259498902
Validation Error: 6.475931639305919

Learning Rate: 0.01, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 2.993880709086672
Validation Error: 2.9946954242174733

Learning Rate: 0.01, # Estimators: 100, Depth: 2
--------------------------------------------------
Training Error: 2.960697750071717
Validation Er

Training Error: 0.10222184461770129
Validation Error: 0.5810409504586627

Learning Rate: 0.3, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 0.7755903637661227
Validation Error: 0.7489373297092125

Learning Rate: 0.3, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 0.6849541330099715
Validation Error: 0.6704786783439802

Learning Rate: 0.3, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 0.5134840315951585
Validation Error: 0.607261137409494

Learning Rate: 0.3, # Estimators: 50, Depth: 10
--------------------------------------------------
Training Error: 0.2442679164955763
Validation Error: 0.6162037373385177

Learning Rate: 0.3, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 0.724601098296789
Validation Error: 0.7005524473535436

Learning Rate: 0.3, # Estimators: 100, Depth: 2
-----------------------

It seems that `learning_rate=0.3`, `n_estimators=500`, and `max_depth=2` provides a very good model

In [83]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor(learning_rate=0.3, n_estimators=500, max_depth=2)
xg_reg.fit(X_train_cv_reg1, y_train1)

train_MSE = calculate_MSE(y_train1, vthreshold_rating(xg_reg.predict(X_train_cv_reg1)))
val_MSE = calculate_MSE(y_val1, vthreshold_rating(xg_reg.predict(X_val_cv_reg1)))

print("Training error based on XGBoost CountVectorizer prediction: %.3f" % train_MSE)
print("Validation error based on XGBoost CountVectorizer prediction: %.3f" % val_MSE)

Training error based on XGBoost CountVectorizer prediction: 0.522
Validation error based on XGBoost CountVectorizer prediction: 0.583


### Meta Models

In this section we look at models that combine both collaborative filtering and language models.

We start by using the predictions of the collaborative filtering as features in our language model

In [104]:
columns_to_keep = ['cleanedPrice', 'isPop', 'isAlternativeRock', 'isJazz', 'isClassical', 'isDanceElectronic', 'reviewWordCount', 'userRating', 'itemRating']
X_train_reg2 = X_train_mod[columns_to_keep]
X_val_reg2 = X_val_mod[columns_to_keep]

In [105]:
min_max_scaler = MinMaxScaler()
X_train_reg2['reviewWordCount'] = X_train_reg2['reviewWordCount'].apply(lambda x: np.log(x))
X_val_reg2['reviewWordCount'] = X_val_reg2['reviewWordCount'].apply(lambda x: np.log(x))

<ipython-input-105-d8a296a7aa94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_reg2['reviewWordCount'] = X_train_reg2['reviewWordCount'].apply(lambda x: np.log(x))
<ipython-input-105-d8a296a7aa94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_reg2['reviewWordCount'] = X_val_reg2['reviewWordCount'].apply(lambda x: np.log(x))


In [106]:
X_train_reg2 = clean_dataset(X_train_reg2)
y_train2 = y_train[y_train.index.isin(X_train_reg2.index)]
X_train2 = X_train[X_train.index.isin(X_train_reg2.index)]

X_val_reg2 = clean_dataset(X_val_reg2)
y_val2 = y_val[y_val.index.isin(X_val_reg2.index)]
X_val2 = X_val[X_val.index.isin(X_val_reg2.index)]

<ipython-input-100-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-100-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [107]:
X_train_reg2 = min_max_scaler.fit_transform(X_train_reg2)
X_val_reg2 = min_max_scaler.transform(X_val_reg2)

In [108]:
alphas = [0.0, 0.01, 0.03, 0.1, 0.3]
for alpha in alphas:
    print("Alpha = {}".format(alpha))
    print("------------")
    reg_model = Ridge(alpha=alpha)
    reg_model.fit(X_train_reg2, y_train2)
    print("Training Error: {}".format(calculate_MSE(y_train2, vthreshold_rating(reg_model.predict(X_train_reg2)))))
    print("Validation Error: {}".format(calculate_MSE(y_val2, vthreshold_rating(reg_model.predict(X_val_reg2)))))
    print()

Alpha = 0.0
------------
Training Error: 0.7891828923294948
Validation Error: 0.9486008427597297

Alpha = 0.01
------------
Training Error: 0.7891677848703272
Validation Error: 0.9484601751838659

Alpha = 0.03
------------
Training Error: 0.7891678465440518
Validation Error: 0.9484577298907713

Alpha = 0.1
------------
Training Error: 0.7891680642355218
Validation Error: 0.948449171698209

Alpha = 0.3
------------
Training Error: 0.7891687018526102
Validation Error: 0.9484247228291963



In [109]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
X_train2['processedReview'] = X_train2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
X_val2['processedReview'] = X_val2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))

<ipython-input-109-25810bf4f6a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train2['processedReview'] = X_train2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
<ipython-input-109-25810bf4f6a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val2['processedReview'] = X_val2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))


In [110]:
cv = CountVectorizer(max_features=1500)
X_train_cv2 = cv.fit_transform(X_train2['processedReview'])
X_val_cv2 = cv.transform(X_val2['processedReview'])

In [111]:
X_train_reg2_sp = sp.csr_matrix(X_train_reg2)
X_train_cv_reg2 = sp.hstack((X_train_cv2, X_train_reg2_sp), format='csr')

X_val_reg2_sp = sp.csr_matrix(X_val_reg2)
X_val_cv_reg2 = sp.hstack((X_val_cv2, X_val_reg2_sp), format='csr')

In [112]:
learning_rates = [0.01, 0.03, 0.1, 0.3, 0.5]
estimators = [50, 100, 200, 500]
depths = [1, 2, 5, 10]

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("--------------------------------------------------")
            xg_reg = XGBRegressor(
                learning_rate=learning_rate, max_depth=depth, n_estimators=estimator)
            xg_reg.fit(X_train_cv_reg2, y_train2)
            print("Training Error: {}".format(calculate_MSE(y_train2, vthreshold_rating(xg_reg.predict(X_train_cv_reg2)))))
            print("Validation Error: {}".format(calculate_MSE(y_val2, vthreshold_rating(xg_reg.predict(X_val_cv_reg2)))))
            print()

Learning Rate: 0.01, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 6.503735872991274
Validation Error: 6.883393063136512

Learning Rate: 0.01, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 6.472925759109321
Validation Error: 6.873810349242472

Learning Rate: 0.01, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 6.415552951975483
Validation Error: 6.766778004848318

Learning Rate: 0.01, # Estimators: 50, Depth: 10
--------------------------------------------------
Training Error: 6.336512333143151
Validation Error: 6.702224040577343

Learning Rate: 0.01, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 2.9116597565047635
Validation Error: 3.2502935567689333

Learning Rate: 0.01, # Estimators: 100, Depth: 2
--------------------------------------------------
Training Error: 2.868043939305317
Validation E

Training Error: 0.06626693654491513
Validation Error: 0.629127338706747

Learning Rate: 0.3, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 0.6695470399035651
Validation Error: 0.8320175368867865

Learning Rate: 0.3, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 0.5877134032041788
Validation Error: 0.7312280170602444

Learning Rate: 0.3, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 0.821128725428028
Validation Error: 0.6644344297462841

Learning Rate: 0.3, # Estimators: 50, Depth: 10
--------------------------------------------------
Training Error: 0.18017706776835557
Validation Error: 0.6767750787866058

Learning Rate: 0.3, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 0.6250522630858881
Validation Error: 0.7799276543267967

Learning Rate: 0.3, # Estimators: 100, Depth: 2
----------------------

It seems `learning_rate=0.3`, `n_estimators=200`, `max_depth=2` performs the best

In [113]:
xg_reg = XGBRegressor(learning_rate=0.3, n_estimators=200, max_depth=2)
xg_reg.fit(X_train_cv_reg2, y_train2)

train_MSE = calculate_MSE(y_train2, vthreshold_rating(xg_reg.predict(X_train_cv_reg2)))
val_MSE = calculate_MSE(y_val2, vthreshold_rating(xg_reg.predict(X_val_cv_reg2)))

print("Training error based on XGBoost CountVectorizer prediction: %.3f" % train_MSE)
print("Validation error based on XGBoost CountVectorizer prediction: %.3f" % val_MSE)

Training error based on XGBoost CountVectorizer prediction: 0.502
Validation error based on XGBoost CountVectorizer prediction: 0.652


This is actually much worse compared to the pure language model.

However, we could also create a meta model by taking a weighted average of predictions from collaborative filtering and the pure language model. We now try this for a few candidate weightings

In [114]:
weights = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

xg_reg = XGBRegressor(learning_rate=0.3, n_estimators=500, max_depth=2)
xg_reg.fit(X_train_cv_reg1, y_train1)

reg_train_preds = vthreshold_rating(xg_reg.predict(X_train_cv_reg1))
reg_val_preds = vthreshold_rating(xg_reg.predict(X_val_cv_reg1))

cf_train_preds = vthreshold_rating(X_train_mod['pred'])
cf_val_preds = vthreshold_rating(X_val_mod['pred'])

for weight in weights:
    print("Weight: %.1f" % weight)
    print("------------")
    train_MSE = calculate_MSE(y_train1, ((weight*reg_train_preds) + ((1.0 - weight)*cf_train_preds)))
    val_MSE = calculate_MSE(y_val1, ((weight*reg_val_preds) + ((1.0 - weight)*cf_val_preds)))
    print("Training error: %.3f" % train_MSE)
    print("Validation error: %.3f" % val_MSE)
    print()

Weight: 0.0
------------
Training error: 0.812
Validation error: 0.950

Weight: 0.1
------------
Training error: 0.754
Validation error: 0.882

Weight: 0.3
------------
Training error: 0.658
Validation error: 0.768

Weight: 0.5
------------
Training error: 0.587
Validation error: 0.681

Weight: 0.7
------------
Training error: 0.542
Validation error: 0.621

Weight: 0.9
------------
Training error: 0.522
Validation error: 0.589

Weight: 1.0
------------
Training error: 0.522
Validation error: 0.583

